In [1]:
%load_ext autoreload
%autoreload 2

import sys
import pandas as pd

sys.path.append('/Users/lorenzostigliano/Documents/University/Imperial/Summer Term/thesis-imperial/src/')
sys.path.append('/Users/lorenzostigliano/Documents/University/Imperial/Summer Term/thesis-imperial/src/utils')
sys.path.append('/Users/lorenzostigliano/Documents/University/Imperial/Summer Term/thesis-imperial/src/models')

from utils.analysis import *
from utils.getters import * 
from utils.plotters import * 
from utils.helpers import * 
from models_config.model_config_GSP_vanilla_exploration import *
from models_config.model_config_GSP import *


import matplotlib.pyplot as plt

plt.rcParams['text.latex.preamble']=[r"\usepackage{lmodern}"]
params = {
    'text.usetex' : True,
    'font.size' : 11,
    'font.family' :'lmodern'
}
plt.rcParams.update(params)


/var/folders/1j/5s36wz6s0jlb1k64vsq1k2zh0000gn/T/ipykernel_26457/735282662.py:21: MatplotlibDeprecationWarning: Support for setting an rcParam that expects a str value to a non-str value is deprecated since 3.5 and support will be removed two minor releases later.
  plt.rcParams['text.latex.preamble']=[r"\usepackage{lmodern}"]


In [2]:
model_teacher = '/Users/lorenzostigliano/Documents/University/Imperial/Summer Term/thesis-imperial/model_data/gender_data/gcn/model_assessment/gcn/models/gcn_MainModel_3Fold_gender_data_gcn_run_3_fixed_init_CV_0_view_0.pt'

model_layer_all_teach = torch.load(model_teacher)


In [4]:
model_student = '/Users/lorenzostigliano/Documents/University/Imperial/Summer Term/thesis-imperial/model_data/gender_data/gcn/model_assessment/gcn_student/models/gcn_student_MainModel_3Fold_gender_data_gcn_student_CV_0_view_0_with_teacher.pt'
model_student_no_kd = '/Users/lorenzostigliano/Documents/University/Imperial/Summer Term/thesis-imperial/model_data/gender_data/gcn/model_assessment/gcn_student/models/gcn_student_MainModel_3Fold_gender_data_gcn_student_run_0_CV_0_view_0.pt'
our_model = '/Users/lorenzostigliano/Documents/University/Imperial/Summer Term/thesis-imperial/model_data/gender_data/gcn/model_assessment/gcn_student_lsp_ensamble_4/models/MainModel_3Fold_gender_data_gcn_student_lsp_ensamble_4_run_0_fixed_init_student_0_CV_0_view_0_alpha_1_beta_2_gamma_2_lambda_1.pt'
lsp_model = '/Users/lorenzostigliano/Documents/University/Imperial/Summer Term/thesis-imperial/model_data/gender_data/gcn/model_assessment/lsp/models/lsp_MainModel_3Fold_gender_data_lsp_run_0_fixed_init_CV_0_view_0_lsp.pt'
fitnet_model = '/Users/lorenzostigliano/Documents/University/Imperial/Summer Term/thesis-imperial/model_data/gender_data/gcn/model_assessment/lsp/models/lsp_MainModel_3Fold_gender_data_lsp_run_0_fixed_init_CV_0_view_0_lsp.pt'
mskd_model = '/Users/lorenzostigliano/Documents/University/Imperial/Summer Term/thesis-imperial/model_data/gender_data/gcn/model_assessment/mskd/models/mskd_MainModel_3Fold_gender_data_mskd_run_0_fixed_init_CV_0_view_0_mskd.pt'

model_layer_all_student = torch.load(model_student)
model_layer_all_student_no = torch.load(model_student_no_kd)
model_layer_our_student = torch.load(our_model)
model_layer_fitnet = torch.load(fitnet_model)
model_layer_lsp = torch.load(lsp_model)
model_layer_mskd = torch.load(mskd_model)

In [40]:
import time
from torch.autograd import Variable

def embeddings(model, model_args):
    
    torch.manual_seed(0)
    np.random.seed(0)
    random.seed(0)   

    model.eval()
    
    G_list = load_data("gender_data", 0, NormalizeInputGraphs=False)

    folds = stratify_splits(G_list, 3)

    graph_embeddings = []
        
    [random.shuffle(folds[i]) for i in range(len(folds))]
    train_set, validation_set, test_set = datasets_splits(folds, model_args, 0)
    train_dataset, val_dataset, threshold_value = model_assessment_split(train_set, validation_set, test_set, model_args)

    for batch_idx, data in enumerate(train_dataset):

        adj = Variable(data['adj'].float(), requires_grad=False).to(device)
        adj = torch.squeeze(adj)

        label = Variable(data['label'].long()).to(device)     

        features = np.identity(adj.shape[0])
        features = Variable(torch.from_numpy(features).float(), requires_grad=False).to("cpu")
        if gcn_student_args["threshold"] in ["median", "mean"]:
            adj = torch.where(adj > threshold_value, torch.tensor([1.0]).to("cpu"), torch.tensor([0.0]).to("cpu"))

        begin_time = time.time()

        x, _ = model(features, adj)
        graph_embeddings.append((x[0].tolist(), label.item()))

    return graph_embeddings

In [41]:
embeddings_student = embeddings(model_layer_all_student)
embeddings_student_no = embeddings(model_layer_all_student_no)
embeddings_teach = embeddings(model_layer_all_teach)
embeddings_our_model = embeddings(model_layer_our_student)
embeddings_fitnet = embeddings(model_layer_fitnet)
embeddings_lsp = embeddings(model_layer_lsp)
embeddings_mskd = embeddings(model_layer_mskd)



TypeError: embeddings() missing 1 required positional argument: 'model_args'

In [44]:
import numpy as np
from sklearn.metrics.pairwise import cosine_distances
from scipy.spatial.distance import euclidean
from scipy.stats import pearsonr

def global_mantel_test(embedding_list1, embedding_list2):
    # Compute cosine distance matrices for both sets of embeddings
    distance_matrix1 = cosine_distances(embedding_list1)
    distance_matrix2 = cosine_distances(embedding_list2)

    # Flatten the distance matrices to 1D arrays
    array1 = distance_matrix1.flatten()
    array2 = distance_matrix2.flatten()

    # Calculate the observed correlation coefficient
    observed_corr, _ = pearsonr(array1, array2)

    return observed_corr

def local_mantel_test(embedding_list1, embedding_list2, partitions):
    # Compute cosine distance matrices for both sets of embeddings
    distance_matrix1 = cosine_distances(embedding_list1)
    distance_matrix2 = cosine_distances(embedding_list2)

    observed_corrs = []

    for partition in partitions:
        # Extract the indices of the data points in the partition
        partition_indices = np.array(partition)

        # Flatten the distance matrices for the partition to 1D arrays
        array1 = distance_matrix1[partition_indices][:, partition_indices].flatten()
        array2 = distance_matrix2[partition_indices][:, partition_indices].flatten()

        # Calculate the observed correlation coefficient for the current partition
        observed_corr, _ = pearsonr(array1, array2)
        observed_corrs.append(observed_corr)

    return observed_corrs

# Function to calculate Euclidean distance between two embeddings
def euclidean_distance(embedding1, embedding2):
    return euclidean(embedding1, embedding2)

# Function to calculate Cosine Similarity between two embeddings
def cosine_similarity(embedding1, embedding2):
    dot_product = np.dot(embedding1, embedding2)
    norm_embedding1 = np.linalg.norm(embedding1)
    norm_embedding2 = np.linalg.norm(embedding2)
    return dot_product / (norm_embedding1 * norm_embedding2)

def get_embedding_stats(embedding_list1, embedding_list2):

    # Calculate Euclidean distance and cosine similarity for each pair of embeddings
    euclidean_distances = [euclidean_distance(e1, e2) for e1, e2 in zip(embedding_list1, embedding_list2)]
    cosine_similarities = [cosine_similarity(e1, e2) for e1, e2 in zip(embedding_list1, embedding_list2)]

    # Perform Global Mantel Test
    observed_corr_global = global_mantel_test(embedding_list1, embedding_list2)

    # Calculate mean values across all embeddings for each matrix
    mean_euclidean_distance = np.mean(euclidean_distances)
    mean_cosine_similarity = np.mean(cosine_similarities)

    # Print the results
    """
    print("Global Mantel Test:")
    print("Observed correlation (Global):", observed_corr_global)
    print("Mean Euclidean distance:", mean_euclidean_distance)
    print("Mean Cosine similarity:", mean_cosine_similarity)
    """
    return mean_euclidean_distance, mean_cosine_similarity, observed_corr_global


In [13]:
# Example embeddings (replace these with your actual data)
embedding_list1 = [np.array(i[0]) for i in embeddings_teach]
embedding_list2 = [np.array(i[0]) for i in embeddings_student_no]
embedding_list3 = [np.array(i[0]) for i in embeddings_student]
embedding_list4 = [np.array(i[0]) for i in embeddings_fitnet]
embedding_list5 = [np.array(i[0]) for i in embeddings_our_model]
embedding_list6 = [np.array(i[0]) for i in embeddings_lsp]
embedding_list7 = [np.array(i[0]) for i in embeddings_mskd]

print("TEACHER")
get_embedding_stats(embedding_list1, embedding_list1)
print("STUDENT")
get_embedding_stats(embedding_list1, embedding_list2)
print("VANILLA")
get_embedding_stats(embedding_list1, embedding_list3)
print("Fitnet")
get_embedding_stats(embedding_list1, embedding_list4)
print("LSP")
get_embedding_stats(embedding_list1, embedding_list6)
print("MSKD")
get_embedding_stats(embedding_list1, embedding_list7)
print("OUR method")
get_embedding_stats(embedding_list1, embedding_list5)


TEACHER
Global Mantel Test:
Observed correlation (Global): 0.9999999999999999
Mean Euclidean distance: 0.0
Mean Cosine similarity: 1.0
STUDENT
Global Mantel Test:
Observed correlation (Global): -0.027315103545876435
Mean Euclidean distance: 1.2779793057221809
Mean Cosine similarity: 0.34330446730593395
VANILLA
Global Mantel Test:
Observed correlation (Global): 0.004066809416458582
Mean Euclidean distance: 1.2897890235619198
Mean Cosine similarity: 0.33460518321984734
Fitnet
Global Mantel Test:
Observed correlation (Global): -0.003479323256800105
Mean Euclidean distance: 1.268201365544475
Mean Cosine similarity: 0.18224965169802876
LSP
Global Mantel Test:
Observed correlation (Global): 0.0039354664294552755
Mean Euclidean distance: 1.2446045790147822
Mean Cosine similarity: 0.19987303028025427
MSKD
Global Mantel Test:
Observed correlation (Global): 0.0034717492406657825
Mean Euclidean distance: 1.2463130796344053
Mean Cosine similarity: 0.12393824992757349
OUR method
Global Mantel Test:

#for all architecture combos:
    # for one model: 
        # we need to loop over runs: 
            # we need to loop over the cross validations:
                # compare embeddings with teacher
                # append scores

    # for all models:
        get average across runs 
        report mean and std for each metric 

In [46]:
def get_embeddings(model_args, model, dataset, run, training_type, cv_n, view, student=-1):

    if model == 'gcn' or model == 'gat':
        model_directory = SAVE_DIR_MODEL_DATA+f'{dataset}/{model_args["backbone"]}/model_assessment/{model}/models/{model}_MainModel_{training_type}_{dataset}_{model}_run_{run}_fixed_init_CV_{cv_n}_view_{view}.pt'        
    
    elif "gcn_student" == model:
        model_directory = SAVE_DIR_MODEL_DATA+f'{dataset}/{model_args["backbone"]}/model_assessment/{model}/models/{model}_MainModel_{training_type}_{dataset}_{model}_run_{run}_fixed_init_CV_{cv_n}_view_{view}.pt'

    elif "gat_student" == model:
        model_directory = SAVE_DIR_MODEL_DATA+f'{dataset}/{model_args["backbone"]}/model_assessment/{model}/models/{model}_MainModel_{training_type}_{dataset}_{model}_run_{run}_fixed_init_CV_{cv_n}_view_{view}.pt'

    elif "teacher" in model:
        model = "_".join(model.split("_")[:2]) 
        model_directory = SAVE_DIR_MODEL_DATA+f'{dataset}/{model_args["backbone"]}/model_assessment/{model}/models/{model}_MainModel_{training_type}_{dataset}_{model}_run_{run}_fixed_init_CV_{cv_n}_view_{view}_with_teacher.pt'

    elif "lsp" == model:
        model_directory = SAVE_DIR_MODEL_DATA+f'{dataset}/{model_args["backbone"]}/model_assessment/{model}/models/lsp_MainModel_{training_type}_{dataset}_{model}_run_{run}_fixed_init_CV_{cv_n}_view_{view}_lsp.pt'

    elif "lsp_gat" == model:
        model_directory = SAVE_DIR_MODEL_DATA+f'{dataset}/{model_args["backbone"]}/model_assessment/{model}/models/lsp_MainModel_{training_type}_{dataset}_{model}_run_{run}_fixed_init_CV_{cv_n}_view_{view}_lsp.pt'

    elif "lsp_gcn" == model:
        model_directory = SAVE_DIR_MODEL_DATA+f'{dataset}/{model_args["backbone"]}/model_assessment/{model}/models/lsp_MainModel_{training_type}_{dataset}_{model}_run_{run}_fixed_init_CV_{cv_n}_view_{view}_lsp.pt'

    elif "mskd" == model:
        model_directory = SAVE_DIR_MODEL_DATA+f'{dataset}/{model_args["backbone"]}/model_assessment/{model}/models/mskd_MainModel_{training_type}_{dataset}_{model}_run_{run}_fixed_init_CV_{cv_n}_view_{view}_mskd.pt'

    elif "mskd_gat" == model:
        model_directory = SAVE_DIR_MODEL_DATA+f'{dataset}/{model_args["backbone"]}/model_assessment/{model}/models/mskd_MainModel_{training_type}_{dataset}_{model}_run_{run}_fixed_init_CV_{cv_n}_view_{view}_mskd.pt'

    elif "mskd_gcn" == model:
        model_directory = SAVE_DIR_MODEL_DATA+f'{dataset}/{model_args["backbone"]}/model_assessment/{model}/models/mskd_MainModel_{training_type}_{dataset}_{model}_run_{run}_fixed_init_CV_{cv_n}_view_{view}_mskd.pt'

    elif "fitnet" == model:
        model_directory = SAVE_DIR_MODEL_DATA+f'{dataset}/{model_args["backbone"]}/model_assessment/{model}/models/fitnet_MainModel_{training_type}_{dataset}_{model}_run_{run}_fixed_init_CV_{cv_n}_view_{view}_fitnet.pt'

    elif "fitnet_gat" == model:
        model_directory = SAVE_DIR_MODEL_DATA+f'{dataset}/{model_args["backbone"]}/model_assessment/{model}/models/fitnet_MainModel_{training_type}_{dataset}_{model}_run_{run}_fixed_init_CV_{cv_n}_view_{view}_fitnet.pt'

    elif "fitnet_gcn" == model:
        model_directory = SAVE_DIR_MODEL_DATA+f'{dataset}/{model_args["backbone"]}/model_assessment/{model}/models/fitnet_MainModel_{training_type}_{dataset}_{model}_run_{run}_fixed_init_CV_{cv_n}_view_{view}_fitnet.pt'

    if "ensamble" in model:
        alpha = str(model_args["alpha"])
        beta = str(model_args["beta"])
        gamma = str(model_args["gamma"])
        lambda_ = str(model_args["lambda"])
        if student == -1:
            model_directory = SAVE_DIR_MODEL_DATA+f'{dataset}/{model_args["backbone"]}/model_assessment/{model}/models/{model}_MainModel_{training_type}_{dataset}_{model}_run_{run}_fixed_init_CV_{cv_n}_view_{view}_alpha_{alpha}_beta_{beta}_gamma_{gamma}_lambda_{lambda_}_T_{T}_{dataset_split}_student_{student}.pickle'        
        else:
            model_directory = SAVE_DIR_MODEL_DATA+f'{dataset}/{model_args["backbone"]}/model_assessment/{model}/models/{model}_MainModel_{training_type}_{dataset}_{model}_run_{run}_fixed_init_CV_{cv_n}_view_{view}_alpha_{alpha}_beta_{beta}_gamma_{gamma}_lambda_{lambda_}_T_{T}_{dataset_split}_student_{student}.pickle'        
    
    model = torch.load(model_directory)
    return [np.array(i[0]) for i in embeddings(model, model_args)] 


In [47]:
get_embeddings(gcn_args, "gcn", dataset="gender_data", run=0, cv_n=0, training_type="3Fold", view=0)

Num training graphs:  466 ; Num test graphs:  232


[array([ 1.98515594, -1.85067332]),
 array([-0.32746908,  0.23258075]),
 array([ 0.05664945, -0.20538005]),
 array([ 0.60868686, -0.20648611]),
 array([-1.30163169,  0.04972421]),
 array([-1.65077937,  2.09063935]),
 array([ 0.21517681, -0.8814562 ]),
 array([ 0.81020474, -0.30202183]),
 array([-0.79672784,  0.08599344]),
 array([-2.34456277,  0.00261824]),
 array([-0.01835469, -1.77421701]),
 array([-1.19097137,  0.22656605]),
 array([-1.66556144,  0.18334949]),
 array([-0.41809025, -0.01132324]),
 array([ 0.16089417, -0.32159927]),
 array([-2.56123877,  1.70089722]),
 array([-2.45744681,  0.23398457]),
 array([-0.18933146,  0.04809058]),
 array([-0.36485168,  0.24443199]),
 array([ 1.84255636, -1.19865608]),
 array([ 0.89417398, -0.84782785]),
 array([-0.62868893,  0.70896196]),
 array([-0.31853917, -0.11256969]),
 array([-0.1724626 , -0.20020649]),
 array([-0.84629977,  1.04442263]),
 array([-0.18179129,  0.13959242]),
 array([-0.06004599, -0.19636229]),
 array([ 0.79940063, -2.3437

In [48]:
def model_embeddings_and_teacher(model_args, model_name, teacher_args, teacher_name, run, training_type, view):

    if training_type == "3Fold":
        cv_number = 3
    if training_type == "5Fold":
        cv_number = 5
    if training_type == "10Fold":
        cv_number = 10  

    e_list, c_list, g_list = [], [], []

    for i in range(cv_number):
        
        model_embeddings = get_embeddings(model_args, model_name, dataset="gender_data", run=run, cv_n=i, training_type=training_type, view=view)
        teacher_embeddings = get_embeddings(teacher_args, teacher_name, dataset="gender_data", run=run, cv_n=i, training_type=training_type, view=view)
        
        e, c, g = get_embedding_stats(model_embeddings, teacher_embeddings)
        e_list.append(e)
        c_list.append(c)
        g_list.append(g)

    return e_list, c_list, g_list


In [49]:
model_args = gcn_args
model_name = "gcn"
teacher_args = gcn_args
teacher_name = "gcn"

all_runs_e = []
all_runs_c = []
all_runs_g = []

for run in range(10):

    e_per_view = []
    c_per_view = []
    g_per_view = []
    
    for view in [0,2,4,5]:

        e_list, c_list, g_list = [], [], []
        
        for training_type in ["3Fold", "5Fold", "10Fold"]:
            
            e_l, c_l, g_l = model_embeddings_and_teacher(model_args, model_name, teacher_args, teacher_name, run, training_type, view)
            e_list.append(e_l)
            c_list.append(c_l)
            g_list.append(g_l)
    
        e_per_view.append(e_list)
        c_per_view.append(c_list)
        g_per_view.append(g_list)
        
    all_runs_e.append(e_per_view)
    all_runs_c.append(c_per_view)
    all_runs_g.append(g_per_view)
    


Num training graphs:  466 ; Num test graphs:  232
Num training graphs:  466 ; Num test graphs:  232
Num training graphs:  466 ; Num test graphs:  232
Num training graphs:  466 ; Num test graphs:  232
Num training graphs:  466 ; Num test graphs:  232
Num training graphs:  466 ; Num test graphs:  232
Num training graphs:  466 ; Num test graphs:  232
Num training graphs:  466 ; Num test graphs:  232
Num training graphs:  466 ; Num test graphs:  232
Num training graphs:  466 ; Num test graphs:  232
Num training graphs:  466 ; Num test graphs:  232
Num training graphs:  466 ; Num test graphs:  232
Num training graphs:  466 ; Num test graphs:  232
Num training graphs:  466 ; Num test graphs:  232
Num training graphs:  466 ; Num test graphs:  232
Num training graphs:  466 ; Num test graphs:  232
Num training graphs:  466 ; Num test graphs:  232
Num training graphs:  466 ; Num test graphs:  232
Num training graphs:  466 ; Num test graphs:  232
Num training graphs:  466 ; Num test graphs:  232
